In [24]:
from huggingface_hub import notebook_login

notebook_login()

In [1]:
%%capture

!pip install faiss-gpu -q

!pip install deep-translator -q

!pip install langdetect -q

!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip uninstall -y torch torchvision torchaudio xformers
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install sentence_transformers peft

!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [2]:
import torch

import numpy as np

import matplotlib.pyplot as plt

import transformers

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

from sentence_transformers import SentenceTransformer

import faiss

from torch.utils.data import Dataset, DataLoader

import torch.nn as nn

from nltk import sent_tokenize

import nltk

from deep_translator import GoogleTranslator

from langdetect import detect

import pickle

import transformers

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

from peft import prepare_model_for_kbit_training
from peft import LoraConfig, TaskType, get_peft_model

from sklearn.model_selection import train_test_split

from unsloth import FastLanguageModel

import json

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
!git lfs install
!rm -rf prikols
!git clone https://github.com/bivba/prikols

Git LFS initialized.


/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Cloning into 'prikols'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 104 (delta 44), reused 56 (delta 26), pack-reused 19 (from 2)
Receiving objects: 100% (104/104), 43.02 MiB | 38.54 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [4]:
token = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device='cuda')

device = 'cuda' if torch.cuda.is_available() else 'cpu'



client = GoogleTranslator(source='ru', target='en')

text = open('/kaggle/working/prikols/context.txt').read()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
#model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit"

torch.cuda.empty_cache()

max_seq_length = 3072

dtype = None

load_in_4bit = True



model, tokenizer = FastLanguageModel.from_pretrained(

    model_name = "bivba/quant_4bit",

    max_seq_length = max_seq_length,

    dtype = dtype,

    load_in_4bit = load_in_4bit

)

==((====))==  Unsloth 2024.12.8: Fast Qwen2 patching. Transformers: 4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/210k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

added_tokens.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/275M [00:00<?, ?B/s]

Unsloth 2024.12.8 patched 48 layers with 48 QKV layers, 48 O layers and 48 MLP layers.


In [6]:
nltk.download('punkt_tab')

def split_document(document, chunk_size=256, overlap=32):

    sentences = sent_tokenize(document)

    chunks = []

    current_chunk = []

    current_length = 0



    for sentence in sentences:

        sentence_length = len(sentence)

        if current_length + sentence_length <= chunk_size:

            current_chunk.append(sentence)

            current_length += sentence_length

        else:

            chunks.append(" ".join(current_chunk))

            current_chunk = current_chunk[-overlap:] + [sentence]

            current_length = sum(len(s) for s in current_chunk)



    if current_chunk:

        chunks.append(" ".join(current_chunk))



    return chunks

[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [7]:
chunks = split_document(text)

encoded = token.encode(chunks, return_numpy=True)

Batches:   0%|          | 0/486 [00:00<?, ?it/s]

In [8]:
faiss_index = faiss.read_index('prikols/context (1).faiss')

def find_context(query, k=2):

  if detect(query) == 'ru':

    query = client.translate(query)

  q_emb = token.encode([query])

  q_emb = q_emb.astype(np.float32)

  _, indices = faiss_index.search(np.array(q_emb), k)

  retrieved_chunks = [chunks[i] for i in indices[0]]

  context = ' '.join(retrieved_chunks)

  return context

In [9]:
from datasets import load_dataset

question_prompt = """You are an AI assistaint helping with answering questions about quantum physics and quantum computer science. Below is the Question, write a response that appropriately answers the question.


### Question:

{}


### Answer:

{}"""



EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

def formatting_prompts_func(examples):

    instructions = [client.translate(example) for example in examples['question']]
    #inputs       = [find_context(instruction) for instruction in instructions]
    outputs      = [client.translate(example) for example in examples['answer']]

    texts = []

    for instruction, input, output in zip(instructions, inputs, outputs):

        # Must add EOS_TOKEN, otherwise your generation will go on forever!

        text = question_prompt.format(instruction, output) + EOS_TOKEN

        texts.append(text)

    return { "text" : texts, }

pass

qas = load_dataset('json', data_files='prikols/test.json', split='train')

Generating train split: 0 examples [00:00, ? examples/s]

In [19]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

test_prompt = """
{}

### Answer:

{}"""


def predict(text):
    with torch.no_grad():
        inputs = tokenizer(
        
        [
        
            question_prompt.format(
        
                client.translate(text),
                
                '', # output - leave this blank for generation!
        
            )
        
        ], return_tensors = "pt").to("cuda")
        
        outputs = model.generate(**inputs, max_new_tokens = 512, use_cache = True, pad_token_id = tokenizer.eos_token_id)
        ans = tokenizer.batch_decode(outputs)
        del inputs
        return ans

In [11]:
import re
def retrieve(answ):
    match = re.search(r"(### Answer:.*?)(?=###|$)", answ[0], re.DOTALL)
    answer = ''
    if match:
        answer = match.group(1)
    
    return answer

In [29]:
from IPython.display import Math

input_message = 'Для реализации протокола В92 абонент А записал случайную бинарную последовательность вида 01001110 и на ее основе создал регистр в соответствии с требованием протокола В92. Отослал кубиты регистра абоненту Б. Абонент Б составил бинарную последовательность 01010100 и действовал на основании этой последовательности на поступающие кубиты операторами, которые определены в протоколе В92 и  проводил их измерение. Далее он сообщил абоненту А  по открытому каналу связи номера кубит, измерение которых дало значение, равное 1. Отметьте ве те секретные ключи, которые могли сформироваться у абонентов? Выполните необходимые преобразования на листе бумаги и отметьте правильный результат.'

Math(retrieve(predict(input_message)))

<IPython.core.display.Math object>

**Evaluating**

In [12]:
!pip install rouge-score -q

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  Preparing metadata (setup.py) ... done


In [25]:
from rouge_score import rouge_scorer

scored = []
answers = []

scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

def compute(reference, predicted):
    scores = scorer.score(reference, predicted)

    scored.append(scores['rougeL'][2])
    answers.append(predicted)

In [26]:
client.translate(qas['question'][16])

'Mark the correct statements for linear and Hermitian operators \\\\\n1) eigenfunctions are real functions \\\\\n2) eigenfunctions form a complete set \\\\\n3) eigenvalues \u200b\u200bform a limited set of ordered numbers \\\\\n4) eigenfunctions are orthonormal \\\\\n5) eigenvalues \u200b\u200bform an equidistant set \\\\\n6) eigenvalues \u200b\u200bare real numbers \\\\\n7) eigenfunctions of commuting operators are the same \\\\'

In [27]:
from tqdm import tqdm


for i in tqdm(range(len(qas))):
    answer = retrieve(predict(qas['question'][i]))
    compute(qas['answer'][i], answer)

100%|██████████| 21/21 [20:29<00:00, 58.55s/it]


In [28]:
with open('/kaggle/working/scores.txt', 'w') as f:
    for n, i in enumerate(scored):
        f.write(str(n + 1) + '.' +str(i) + '\n')

In [29]:
with open('/kaggle/working/answers.txt', 'w') as f:
    for n, i in enumerate(answers):
        f.write(str(n + 1) + '.' + str(i) + '\n')

In [30]:
%cd /kaggle/working
from IPython.display import FileLink
FileLink('answers.txt')

/kaggle/working


/kaggle/working/answers.txt

In [23]:
from IPython.display import FileLink
FileLink('scores.txt')

/kaggle/working/scores.txt

In [17]:
torch.cuda.empty_cache()